Question 18

3D printer (PPP)

In [ ]:
import numpy as np

def manipulator_jacobian(num_links, dh_parameters):

    jacobian = np.zeros((6, num_links))


    q = [0] * num_links
    d = [0] * num_links

    for i in range(num_links):
        theta, d_i, a, alpha = dh_parameters[i]

        #  DH transformation matrix
        cos_theta = np.cos(theta)
        sin_theta = np.sin(theta)
        cos_alpha = np.cos(alpha)
        sin_alpha = np.sin(alpha)

        transformation_matrix = np.array([
            [cos_theta, -sin_theta * cos_alpha, sin_theta * sin_alpha, a * cos_theta],
            [sin_theta, cos_theta * cos_alpha, -cos_theta * sin_alpha, a * sin_theta],
            [0, sin_alpha, cos_alpha, d_i],
            [0, 0, 0, 1]
        ])

        # screw axis for the joint
        z_i = transformation_matrix[:3, 2]
        p_i = transformation_matrix[:3, 3]

        if i == 0:
            jacobian[:3, i] = np.cross([0, 0, 1], p_i)
            jacobian[3:, i] = [0, 0, 1]
        else:
            jacobian[:3, i] = np.cross(z_i, p_i - p_prev)
            jacobian[3:, i] = z_i

        p_prev = p_i

    return jacobian

def forward_kinematics(num_links, dh_parameters):
    # Initialize the transformation matrix
    T = np.eye(4)

    for i in range(num_links):
        theta, d_i, a, alpha = dh_parameters[i]

        # DH transformation matrix
        cos_theta = np.cos(theta)
        sin_theta = np.sin(theta)
        cos_alpha = np.cos(alpha)
        sin_alpha = np.sin(alpha)

        transformation_matrix = np.array([
            [cos_theta, -sin_theta * cos_alpha, sin_theta * sin_alpha, a * cos_theta],
            [sin_theta, cos_theta * cos_alpha, -cos_theta * sin_alpha, a * sin_theta],
            [0, sin_alpha, cos_alpha, d_i],
            [0, 0, 0, 1]
        ])

        # overall transformation matrix
        T = np.dot(T, transformation_matrix)

    return T

#  DH parameters for the PPP configuration
d1 = 2.0
d2 = 3.0
d3 = 4.0

dh_params = np.array([[0, d1, 0, 0],
                      [0, d2, 0, 0],
                      [0, d3, 0, 0]])

# forward kinematics for the given DH parameters
end_effector_pose = forward_kinematics(3, dh_params)


print("End-effector position:")
print(end_effector_pose[:3, 3])

# matrix for the given DH parameters
Jacobian = manipulator_jacobian(3, dh_params)

#joint velocities (for example, in mm/s)
d1_dot = 0.1
d2_dot = 0.2
d3_dot = 0.3
joint_velocities = np.array([d1_dot, d2_dot, d3_dot])

# the end-effector velocity using the Jacobian
end_effector_velocity = np.dot(Jacobian, joint_velocities)

print("End-effector velocity:")
print(end_effector_velocity)


End-effector position:
[0. 0. 9.]
End-effector velocity:
[0.  0.  0.  0.  0.  0.6]


Question(17)

Python subroutine for the inverse kinematics of the spherical wrist

In [ ]:
import math

def spherical_wrist_inverse_kinematics(x, y, z, link_lengths):
    #link lengths
    l1, l2, l3 = link_lengths

    #first joint angle (q1)
    q1 = math.atan2(y, x)

    # distance from the base to the projection of the end-effector in the xy-plane
    D_xy = math.sqrt(x**2 + y**2)

    #height difference between the end-effector and the first wrist joint
    delta_z = z - l1

    #distance between the first wrist joint and the end-effector in the xy-plane
    D = math.sqrt(D_xy**2 + delta_z**2)

    #angles required to form a right triangle with sides l2 and l3
    alpha = math.acos((l2**2 + l3**2 - D**2) / (2 * l2 * l3))
    beta = math.atan2(delta_z, D_xy)

    #second joint angle (q2)
    q2 = math.pi/2 - alpha - beta

    #third joint angle (q3)
    q3 = math.atan2(l3 * math.sin(alpha), l2 + l3 * math.cos(alpha))

    return q1, q2, q3





Joint Angles (q1, q2, q3): (0.0, 1.4487583923385514, 0.4424716810880932)


Question 14

python subroutine for the joint velocities using
end-effector cartesian velocities.

In [ ]:
import numpy as np

def joint_velocities(Jacobian,end_effector_velocity):

  Inerse_Jacobian = np.linalg.piv(Jacobian)

  joint_velocities = np.dot(Inerse_Jacobian,end_effector_velocity)

  return joint_velocities




 Question (12)

 python subroutine to solve for the inverse position kinematics for
 the Stanford manipulator

In [ ]:
import math
import numpy as np

def stanford_manipulator(x,y,z,r,d3,a2,s):
# d3 = linear distance of prismatic joint
  # r= resulatant of position of x and y

  r = x**2 + y**2

  s = z - d1
  theta1 = np.pi + np.arctan(x/y)

  theta2 = np.pi - np.arctan(r/s)

  d3 = math.sqrt(r**2 + s**2) - a2

  return theta1, theta2, d3

  # example
  x = 2
  y = 3
  z = 3
  d1 = 2
  a2 = 3

  theta1, theta2, d3 = stanford_manipulator(x,y,z,r,d3,a2,s)

  print(theta1, theta2, d3)




Question (13)

python subroutine for the inverse position kinematics for
the SCARA manipulator

In [ ]:
import numpy as np

def inverse_kinematics_scara(x, y, z, l1, l2):

  r = np.sqrt(x**2 + y**2)

  theta1 = np.arctan2(y, x)

  theta2 = np.arccos((r**2 + l1**2 - l2**2) / (2 * l1 * r))

  theta3 = np.arctan2(z, r)

  return theta1, theta2, theta3

# Example
l1 = 2
l2 = 2
x = 1.5
y = 2
z = 3

theta1, theta2, theta3 = inverse_kinematics_scara(x, y, z, l1, l2)

print(theta1, theta2, theta3)



0.9272952180016122 0.895664793857865 0.8760580505981934


Question 11


In [3]:
from sympy import symbols, Eq, solve

# Define symbolic variables
q1, q2, q3 = symbols('q1 q2 q3')
dq1, dq2, dq3 = symbols('dq1 dq2 dq3')
ddq1, ddq2, ddq3 = symbols('ddq1 ddq2 ddq3')
Tau1, Tau2, Tau3 = symbols('Tau1 Tau2 Tau3')

# Define D(q) and V(q, dq) for your specific robot (replace the placeholders)
# Example D(q) for a 3-DOF robot (a diagonal mass matrix)
m1, m2, m3 = symbols('m1 m2 m3')  # Masses of the links
D_q = [[m1, 0, 0],
       [0, m2, 0],
       [0, 0, m3]]

# Example V(q, dq) for a 3-DOF robot (ignoring Coriolis and centrifugal effects)
V_q_dq = [0, 0, 0]

# Step 2: Calculate ddq
eqn1 = Eq(D_q[0][0] * ddq1 + D_q[0][1] * ddq2 + D_q[0][2] * ddq3 + V_q_dq[0] - Tau1, 0)
eqn2 = Eq(D_q[1][0] * ddq1 + D_q[1][1] * ddq2 + D_q[1][2] * ddq3 + V_q_dq[1] - Tau2, 0)
eqn3 = Eq(D_q[2][0] * ddq1 + D_q[2][1] * ddq2 + D_q[2][2] * ddq3 + V_q_dq[2] - Tau3, 0)

# Step 3: Solve for ddq
solution = solve((eqn1, eqn2, eqn3), (ddq1, ddq2, ddq3))

